[![Binder](https://mybinder.org/badge_logo.svg)](https://nbviewer.org/github/Sistemas-Multimedia/Sistemas-Multimedia.github.io/blob/master/milestones/09-ME/full_search_dense_ME.ipynb)

# Full search dense (1x1) ME

If we extrapolate the idea used in `full_search_block_based_BE()` when the `block_side==1`, there are a huge amount of redundant substractions when adjacent blocks are processed. In the following implementation, the subtractions are performed only once because all the "blocks" (that in this case are of one pixel) are subtracted at each seaching point (one for search area). Another characteristic of this algorithm is that the error frames are smoothed to increase the correlation between the motion vectors.

In [ ]:
import numpy as np
import image_3
import YCoCg as YUV
import motion
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['text.usetex'] = True
!ln -sf ~/MRVC/src/LP.py .
import LP
import cv2 as cv
!ln -sf ~/quantization/information.py .
import information

## Testing with moving circles (max_abs_motion=1)

In [ ]:
R = image_3.read("/home/vruiz/MRVC/sequences/moving_circles/", 0)
image_3.show(R, 'Reference')
R_Y = YUV.from_RGB(R.astype(np.int16))[...,0]
P = image_3.read("/home/vruiz/MRVC/sequences/moving_circles/", 1)
image_3.show(P, 'Predicted')
P_Y = YUV.from_RGB(P.astype(np.int16))[...,0]

The top circle moves to the right and the bottom circle moves to the left. Therefore, if we want to generate the predicted frame (bottom) from the reference one (top), all the top MVs (Motion Vectors) related to the circle should be (x=1, y=0), and all the bottom MVs (-1, 0).

In [ ]:
MVs = motion.full_search_dense_ME(P_Y, R_Y, 16, 17)

In [ ]:
motion.show_vectors(MVs)

In [ ]:
hatP = motion.make_prediction(R, MVs.astype(np.float32))

In [ ]:
image_3.show(R, "reference ${\mathbf R}$")
image_3.show(P, "predicted ${\mathbf P}$")
image_3.show(hatP, "prediction $\hat{\mathbf P}$")
image_3.show(image_3.normalize(hatP - P), "prediction error")

## Testing with moving circles (max_abs_motion=2)

In [ ]:
R = image_3.read("/home/vruiz/MRVC/sequences/moving_circles/", 0)
R_Y = YUV.from_RGB(R.astype(np.int16))[...,0]
P = image_3.read("/home/vruiz/MRVC/sequences/moving_circles/", 2)
P_Y = YUV.from_RGB(P.astype(np.int16))[...,0]

In [ ]:
MVs = motion.full_search_dense_ME(P_Y, R_Y, 16, 17)

In [ ]:
motion.show_vectors(MVs)

In [ ]:
hatP = motion.make_prediction(R, MVs.astype(np.float32))

In [ ]:
image_3.show(R, "reference ${\mathbf R}$")
image_3.show(P, "predicted ${\mathbf P}$")
image_3.show(hatP, "prediction $\hat{\mathbf P}$")
image_3.show(P - hatP + 128).astype(np.uint8), "prediction error ($P-\hat{P}$)")

## Testing with mobile

In [ ]:
R = image_3.read("/home/vruiz/MRVC/sequences/mobile/", 0)
R_Y = YUV.from_RGB(R.astype(np.int16))[...,0]
P = image_3.read("/home/vruiz/MRVC/sequences/mobile/", 1)
P_Y = YUV.from_RGB(P.astype(np.int16))[...,0]

### First, without ME

In [ ]:
image_3.show(R, "reference ${\mathbf R}$")
P_entropy = information.entropy(P.flatten())
image_3.show(P, "predicted ${\mathbf P}$" + f" entropy={P_entropy:1.2f} bits/component")
P_R = np.clip(P.astype(np.int16) - R + 128, 0, 255)
P_R_entropy = information.entropy(P_R.flatten())
image_3.show(P_R.astype(np.uint8), "(No ME) $({\mathbf P} - {\mathbf R})$" + f" entropy={P_R_entropy:1.2f} bits/component")

### Second, with ME

In [ ]:
MVs = motion.full_search_dense_ME(P_Y, R_Y, 16, 17)

In [ ]:
MVs_entropy = information.entropy(MVs.flatten())

In [ ]:
motion.show_vectors(MVs[::10, ::10], title="${\mathbf V}$" + f", 1x1 ME" + f", entropy={MVs_entropy:1.2f} bits/component" +  f", {MVs.shape[0]}x{MVs.shape[1]} vectors")

In [ ]:
hatP = motion.make_prediction(R, MVs.astype(np.float32))

In [ ]:
#image_3.show(R, "reference ${\mathbf R}$")
#image_3.show(P, "predicted ${\mathbf P}$" + f", entropy={P_entropy:1.2f} bits/component")
image_3.show(hatP, "prediction $\hat{\mathbf P}$")
P_hatP = P - hatP
P_hatP_entropy = information.entropy(P_hatP.flatten())
image_3.show((P_hatP + 128).astype(np.uint8), "${\mathbf P} - \hat{\mathbf P}$" + f", entropy={P_hatP_entropy:1.2f} bits/component")

In [ ]:
print("Entropy of the predicted frame:                 ", f"{P_entropy:1.3f}", "bits/component")
print("Entropy of the residue frame without ME:        ", f"{P_R_entropy:1.3f}", "bits/component")
print("Entropy of the residue frame using ME:          ", f"{P_hatP_entropy:1.3f}", "bits/component")
print("Entropy reduction in the texture without ME:    ", f"{P_entropy - P_R_entropy:1.3f}", "bits/component")
print("Entropy reduction in the texture using ME:      ", f"{P_entropy - P_hatP_entropy:1.3f}", "bits/component")
print("Entropy of the components of the motion vectors:", f"{MVs_entropy:1.3f}", "bits/component")
total_entropy = P_hatP_entropy + MVs_entropy
print("Entropy the texture + motion vectors:           ", f"{total_entropy:1.3f}", "bits/component")

## Testing with bus
TODO: compute entropies.

In [ ]:
R = image_3.read("/home/vruiz/MRVC/sequences/bus/", 0)
R_Y = YUV.from_RGB(R.astype(np.int16))[...,0]
P = image_3.read("/home/vruiz/MRVC/sequences/bus/", 1)
P_Y = YUV.from_RGB(P.astype(np.int16))[...,0]

In [ ]:
MVs = motion.full_search_dense_ME(P_Y, R_Y, 16, 17)

In [ ]:
motion.show_vectors(MVs[::10, ::10])

In [ ]:
hatP = motion.make_prediction(R, MVs.astype(np.float32))

In [ ]:
image_3.show(R, "reference ${\mathbf R}$")
image_3.show(P, "predicted ${\mathbf P}$")
image_3.show(hatP, "prediction $\hat{\mathbf P}$")
image_3.show((hatP - P + 128).astype(np.uint8), "prediction error")

## Testing with a tile of stockholm
TODO: compute entropies.

In [ ]:
prefix = "/home/vruiz/MRVC/sequences/stockholm/"
R = image_3.read(prefix, 0)[100:356,100:612]
P = image_3.read(prefix, 1)[100:356,100:612]
R_Y = YUV.from_RGB(R.astype(np.int16))[...,0]
P_Y = YUV.from_RGB(P.astype(np.int16))[...,0]

In [ ]:
MVs = motion.full_search_dense_ME(P_Y, R_Y, 16, 17)

In [ ]:
entropy = information.entropy(MVs.flatten())
motion.show_vectors(MVs[::10, ::10], title="${\mathbf V}$ (1x1 ME) " + f"entropy={entropy:1.2f} bits/component" + f", {MVs.shape[0]}x{MVs.shape[1]} vectors")

In [ ]:
hatP = motion.make_prediction(R, MVs.astype(np.float32))

In [ ]:
image_3.show(R, "reference ${\mathbf R}$")
image_3.show(P, "predicted ${\mathbf P}$")
image_3.show(hatP, "prediction $\hat{\mathbf P}$")
P_hatP = P - hatP
entropy = information.entropy(P_hatP.flatten())
image_3.show((P_hatP + 128).astype(np.uint8), "${\mathbf P}$ - $\hat{\mathbf P}$ (1x1 ME)" + f" entropy={entropy:1.2f} bits/component")

## Ignore the rest

In [ ]:
########################################################################################

In [ ]:
R = YUV.from_RGB(frame.read("/home/vruiz/MRVC/sequences/bus/", 0).astype(np.int16))
P = YUV.from_RGB(frame.read("/home/vruiz/MRVC/sequences/bus/", 1).astype(np.int16))

In [ ]:
MVs = full_search_dense_ME(P[...,0], R[...,0], 32, 17)

In [ ]:
R = frame.read("/home/vruiz/MRVC/sequences/bus/", 0)
P = frame.read("/home/vruiz/MRVC/sequences/bus/", 1)
prediction = motion.make_prediction(R, MVs.astype(np.float32))
show_frame(prediction, "Prediction")
show_frame(P, "Predicted")
show_frame(normalize(prediction-P), "error")

In [ ]:
R = YUV.from_RGB(frame.read("/home/vruiz/MRVC/sequences/bus/", 0).astype(np.int16))
P = YUV.from_RGB(frame.read("/home/vruiz/MRVC/sequences/bus/", 1).astype(np.int16))

In [ ]:
MVs = full_search_dense_ME(P[...,0], R[...,0], 32, 17)

In [ ]:
R = frame.read("/home/vruiz/MRVC/sequences/bus/", 0)
P = frame.read("/home/vruiz/MRVC/sequences/bus/", 1)
prediction = motion.make_prediction(R, MVs.astype(np.float32))
show_frame(prediction, "Prediction")
show_frame(P, "Predicted")
show_frame(normalize(prediction-P), "error")

In [ ]:
R = YUV.from_RGB(frame.read("/home/vruiz/MRVC/sequences/stockholm/", 0).astype(np.int16))
P = YUV.from_RGB(frame.read("/home/vruiz/MRVC/sequences/stockholm/", 1).astype(np.int16))

In [ ]:
MVs = full_search_dense_ME(P[...,0], R[...,0], 32, 17)

In [ ]:
R = frame.read("/home/vruiz/MRVC/sequences/stockholm/", 0)
P = frame.read("/home/vruiz/MRVC/sequences/stockholm/", 1)
prediction = motion.make_prediction(R, MVs.astype(np.float32))
show_frame(prediction, "Prediction")
show_frame(P, "Predicted")
show_frame(normalize(prediction-P), "error")

In [ ]:
# Copiado common.py Z_interpolation

# (x, y)
#      5       3       8
#      1       0       2
#      7       4       6
# (-1,-1) ( 0,-1) ( 1,-1)
# (-1, 0) ( 0, 0) ( 1, 0)
# (-1, 1) ( 0, 1) ( 1, 1)
spiral  = np.array([( 0, 0), (-1, 0), ( 1, 0), ( 0,-1), ( 0, 1), (-1,-1), ( 1, 1), (-1, 1), ( 1,-1)], dtype=np.int16)
#                        0        1        2        3        4        5        6        7        8

# (0,0) (1,0) (2,0)
# (0,1) (1,1) (2,1)
# (0,2) (1,2) (2,2)
spiral_ = np.array([( 1, 1), ( 0, 1), ( 2, 1), ( 1, 0), ( 1, 2), ( 0, 0), ( 2, 2), ( 0, 2), ( 2, 0)], dtype=np.int16)

def spiral_estimate(P, R, window_side=15):
    print("window_side =", window_side)
    extended_R = np.zeros((R.shape[0]+2, R.shape[1]+2))
    extended_R[1:R.shape[0]+1, 1:R.shape[1]+1] = R
    print("extended_R.shape", extended_R.shape)
    #extended_R = np.pad(R, (1, 1), 'edge')
    #print("extended_R.shape", extended_R.shape)
    D = np.zeros((P.shape[0], P.shape[1], 9), dtype=np.float32)
    for i in range(9):
        diff = extended_R[spiral_[i][1] : R.shape[0] + spiral_[i][1],
                          spiral_[i][0] : R.shape[1] + spiral_[i][0]] - P
        D[:,:,i] = diff*diff
    # Tenemos la distancia L2, pixel-wise, para cada posible desplazamiento de ambas matrices, R y P.
    # Ahora se trata de encontrar una nueva matriz de distancias donde se tenga en cuenta las distancias adyacentes.
    DD = np.empty_like(D).astype(np.float32)
    DD[...] = D
    kernel = np.ones((window_side,window_side),np.float32)/(window_side*window_side)
    D = cv.filter2D(DD, -1, kernel)
    #show_frame(D[...,0]+D[...,1])
    #show_frame(D[...,0])
    #for y in range(D.shape[0]):
    #    for x in range(D.shape[1]):
    #        print(D[y,x], end=' ')
    #    print()
    #print(80*"=")
    flow_indexes = np.argmin(np.rint(D), axis=2)
    flow = spiral[flow_indexes]
    return flow
    
def local_estimate(P, R, window_side=15):
    flow = np.zeros((R.shape[0], R.shape[1], 2), dtype=np.int16)
    flow = spiral_estimate(P, R, window_side=window_side)
    return flow

In [ ]:
R = frame.read("../sequences/moving_circles/", 0)
P = frame.read("../sequences/moving_circles/", 1)
R_Y = YUV.from_RGB(R.astype(np.int16))[...,0]
P_Y = YUV.from_RGB(P.astype(np.int16))[...,0]

In [ ]:
MVs = local_estimate(P_Y, R_Y)

In [ ]:
show_frame(MVs[...,0])

In [ ]:
prediction = motion.make_prediction(R, MVs)
show_frame(prediction)
show_frame(P)

In [ ]:
show_frame(P-prediction)

In [ ]:
R = YUV.from_RGB(frame.read("../sequences/moving_circles/", 0).astype(np.int16))[...,0]
P = YUV.from_RGB(frame.read("../sequences/moving_circles/", 2).astype(np.int16))[...,0]

In [ ]:
flow = local_estimate(P, R)

In [ ]:
show_frame(flow[...,0])

In [ ]:
R = frame.read("../sequences/moving_circles/", 0)
P = frame.read("../sequences/moving_circles/", 2)
prediction = motion.make_prediction(R, flow)
show_frame(prediction)
show_frame(P)

In [ ]:
show_frame(P-prediction)

In [ ]:
########### Antiguo 

In [ ]:
R = YUV.from_RGB(frame.read("../sequences/moving_circles/", 0).astype(np.int16))[...,0]
P = YUV.from_RGB(frame.read("../sequences/moving_circles/", 2).astype(np.int16))[...,0]

In [ ]:
#      5       3       8
#      1       0       2
#      7       4       6
# (-1,-1) (-1, 0) (-1, 1)
# ( 0,-1) ( 0, 0) ( 0, 1)
# ( 1,-1) ( 1, 0) ( 1, 1)
#spiral  = np.array([(-1,-1), (-1, 1), ( 1,-1), ( 1, 1), (-1, 0), ( 1, 0), ( 0,-1), ( 0, 1), ( 0, 0)])
spiral  = np.array([( 0, 0), (-1,-1), (-1, 1), ( 1,-1), ( 1, 1), (-1, 0), ( 1, 0), ( 0,-1), ( 0, 1)])
spiral  = np.array([( 0, 0), ( 0,-1), ( 0, 1), (-1, 0), ( 1, 0), (-1,-1), ( 1, 1), ( 1,-1), (-1, 1)])

# (0,0) (0,1) (0,2)
# (1,0) (1,1) (1,2)
# (2,0) (2,1) (2,2)
spiral_ = np.array([( 1, 1), ( 0, 0), ( 0, 2), ( 2, 0), ( 2, 2), ( 0, 1), ( 2, 1), ( 1, 0), ( 1, 2)])
spiral_ = np.array([( 1, 1), ( 1, 0), ( 1, 2), ( 0, 1), ( 2, 1), ( 0, 0), ( 2, 2), ( 2, 0), ( 0, 2)])

In [ ]:
# (x, y)
#      5       3       8
#      1       0       2
#      7       4       6
# (-1,-1) ( 0,-1) ( 1,-1)
# (-1, 0) ( 0, 0) ( 1, 0)
# (-1, 1) ( 0, 1) ( 1, 1)
spiral  = np.array([( 0, 0), (-1, 0), ( 1, 0), ( 0,-1), ( 0, 1), (-1,-1), ( 1, 1), (-1, 1), ( 1,-1)], dtype=np.int16)
#                        0        1        2        3        4        5        6        7        8

# (0,0) (1,0) (2,0)
# (0,1) (1,1) (2,1)
# (0,2) (1,2) (2,2)
spiral_ = np.array([( 1, 1), ( 0, 1), ( 2, 1), ( 1, 0), ( 1, 2), ( 0, 0), ( 2, 2), ( 0, 2), ( 2, 0)], dtype=np.int16)

In [ ]:
print(spiral.dtype)

In [ ]:
extended_R = np.zeros((R.shape[0]+2, R.shape[1]+2))
extended_R[1:R.shape[0]+1, 1:R.shape[1]+1] = R

In [ ]:
#extended_R = np.pad(R, (1, 1), 'edge') # Ojo, no funciona

In [ ]:
show_frame(extended_R)

In [ ]:
D = np.zeros((P.shape[0], P.shape[1], 9), dtype=np.float32)

In [ ]:
for i in range(9):
    diff = extended_R[spiral_[i][1]:R.shape[0]+spiral_[i][1], spiral_[i][0]:R.shape[1]+spiral_[i][0]] - P
    D[:,:,i] = diff*diff

In [ ]:
print(D[0,0,0], D[0,0,1], D[0,0,2], D[0,0,3])

In [ ]:
DD = np.empty_like(D).astype(np.float32)
DD[...] = D
print(D[0,0,1])
show_frame(D[...,0])
kernel = np.ones((15,15),np.float32)/(15*15)
D = cv.filter2D(DD,-1,kernel)
#show_frame(D[...,0]+D[...,1])
show_frame(np.rint(D[...,0]))


In [ ]:
flow_indexes = np.argmin(np.rint(D), axis=2)
for i in range(64):
    for j in range(64):
        print(flow_indexes[i,j], end='')
    print()

In [ ]:
show_frame(flow_indexes)

In [ ]:
print(D[0,0,0], D[0,0,1], D[0,0,2], D[0,0,3])

In [ ]:
flow = spiral[flow_indexes]

In [ ]:
show_frame(D[:,:,1] - D[:,:,8])

In [ ]:
R = frame.read("../sequences/moving_circles/", 0)

In [ ]:
show_frame(R)

In [ ]:
prediction = motion.make_prediction(R, flow.astype(np.float32))

In [ ]:
show_frame(prediction)

In [ ]:
P = frame.read("../sequences/moving_circles/", 1)
show_frame(P)

In [ ]:
R = YUV.from_RGB(frame.read("../sequences/moving_circles/", 0).astype(np.int16))
P = YUV.from_RGB(frame.read("../sequences/moving_circles/", 2).astype(np.int16))

In [ ]:
def spiral_estimate(P, R, window_side=15):
    print("window_side =", window_side)
    extended_R = np.zeros((R.shape[0]+2, R.shape[1]+2))
    extended_R[1:R.shape[0]+1, 1:R.shape[1]+1] = R
    print("extended_R.shape", extended_R.shape)
    #extended_R = np.pad(R, (1, 1), 'edge')
    #print("extended_R.shape", extended_R.shape)
    D = np.zeros((P.shape[0], P.shape[1], 9), dtype=np.float32)
    for i in range(9):
        diff = extended_R[spiral_[i][1] : R.shape[0] + spiral_[i][1],
                          spiral_[i][0] : R.shape[1] + spiral_[i][0]] - P
        D[:,:,i] = diff*diff
    # Tenemos la distancia L2, pixel-wise, para cada posible desplazamiento de ambas matrices, R y P.
    # Ahora se trata de encontrar una nueva matriz de distancias donde se tenga en cuenta las distancias adyacentes.
    DD = np.empty_like(D).astype(np.float32)
    DD[...] = D
    kernel = np.ones((window_side,window_side),np.float32)/(window_side*window_side)
    D = cv.filter2D(DD, -1, kernel)
    #show_frame(D[...,0]+D[...,1])
    #show_frame(D[...,0])
    #for y in range(D.shape[0]):
    #    for x in range(D.shape[1]):
    #        print(D[y,x], end=' ')
    #    print()
    #print(80*"=")
    flow_indexes = np.argmin(np.rint(D), axis=2)
    for i in range(flow_indexes.shape[0]):
        for j in range(flow_indexes.shape[1]):
            print(flow_indexes[i,j], end='')
        print()
    print(80*"=")
    flow = spiral[flow_indexes]
    return flow

In [ ]:
flow = spiral_estimate(P[...,0], R[...,0])

In [ ]:
# Funciona para log2_search_area = 0 y 1 (search_area 1 y 2)
def estimate(P, R, log2_search_area, n_levels):
    if log2_search_area > 0:
        P_pyramid = LP.analyze(P, log2_search_area)
        R_pyramid = LP.analyze(R, log2_search_area)
    else:
        P_pyramid = [P]
        R_pyramid = [R]
    #total_flow = np.zeros(((R_pyramid[0].shape[0]+1)//2, (R_pyramid[0].shape[1]+1)//2, 2), dtype=np.int16)
    total_flow = np.zeros((R_pyramid[0].shape[0], R_pyramid[0].shape[1], 2), dtype=np.int16)
    for l in range(n_levels):
        _R_pyramid = motion.make_prediction(R_pyramid[l], total_flow.astype(np.float32))
        flow = spiral_estimate(P_pyramid[l], _R_pyramid)
        total_flow *= 2
        total_flow += flow
        total_flow = total_flow.astype(np.int16)
        #for y in range(total_flow.shape[0]):
        #    for x in range(total_flow.shape[1]):
        #        print(total_flow[y,x], end=' ')
        #    print()
        #print(80*"=")
        #show_frame(R_pyramid[l+1])
        R_pyramid[l+1] = LP.synthesize_step(R_pyramid[l], R_pyramid[l+1])
        P_pyramid[l+1] = LP.synthesize_step(P_pyramid[l], P_pyramid[l+1])
        total_flow = LP.synthesize_step(total_flow.astype(np.float32), np.zeros(total_flow.shape)).astype(np.int16)
    _R = motion.make_prediction(R, total_flow.astype(np.float32))
    flow = spiral_estimate(P, _R)
    if log2_search_area > 0:
        #total_flow = LP.synthesize_step(total_flow.astype(np.float32), np.zeros(total_flow.shape)).astype(np.int16)
        #total_flow *= 2
        total_flow += flow
        total_flow = total_flow.astype(np.int16)
    else:
        return flow
    return total_flow

In [ ]:
flow = estimate(P[...,0], R[...,0], 0, 0)

In [ ]:
def local_estimate(P, R):
    flow = np.zeros((R.shape[0], R.shape[1], 2), dtype=np.int16)
    flow = spiral_estimate(P, R)
    return flow

In [ ]:
local_estimate(P[...,0], R[...,0])

In [ ]:
def spiral_estimate2(initial_flow, P, R, window_side=15):
    print("window_side =", window_side)
    print("inital_flow.dtype", initial_flow.dtype)
    print("R.shape", R.shape)
    print("initial_flow.shape", initial_flow.shape)
    extended_R = np.zeros((R.shape[0]+2, R.shape[1]+2))
    extended_R[1:R.shape[0]+1, 1:R.shape[1]+1] = motion.make_prediction(R, initial_flow)[:R.shape[0], :R.shape[1]]
    show_frame(R, "R")
    show_frame(extended_R[1:R.shape[0]+1, 1:R.shape[1]+1], "extended_R")
    show_frame(P, "P")
    D = np.zeros((P.shape[0], P.shape[1], 9), dtype=np.float32)
    for i in range(9):
        diff = extended_R[spiral_[i][1] : R.shape[0] + spiral_[i][1],
                          spiral_[i][0] : R.shape[1] + spiral_[i][0]] - P
        #print(diff)
        D[:,:,i] = abs(diff)#diff*diff
    # Tenemos la distancia L2, pixel-wise, para cada posible desplazamiento de ambas matrices, R y P.
    # Ahora se trata de encontrar una nueva matriz de distancias donde se tenga en cuenta las distancias adyacentes.
    DD = np.empty_like(D).astype(np.float32)
    DD[...] = D
    #kernel = np.ones((window_side,window_side),np.float32)/(window_side*window_side)
    #D = cv.filter2D(DD, -1, kernel).astype(np.int)
    print(DD.shape, window_side)
    D = cv.GaussianBlur(DD, (window_side, window_side), 0).astype(np.int)
    #show_frame(D[...,0]+D[...,1])
    #show_frame(D[...,0])
    #for y in range(D.shape[0]):
    #    for x in range(D.shape[1]):
    #        print(D[y,x], end=' ')
    #    print()
    #print(80*"=")
    #for i in range(9):
    #    print(D[...,i])
    flow_indexes = np.argmin(D, axis=2)
    #for i in range(flow_indexes.shape[0]):
    #    for j in range(flow_indexes.shape[1]):
    #        print(flow_indexes[i,j], end='')
    #    print()
    #print(80*"=")
    #print(spiral[flow_indexes].shape)
    #print(initial_flow.shape)
    flow = initial_flow[:flow_indexes.shape[0], :flow_indexes.shape[1]] + spiral[flow_indexes]
    return flow

In [ ]:
flow = np.zeros((R.shape[0], R.shape[1], 2), dtype=np.int16)
print("flow.shape", flow.shape)
flow = spiral_estimate2(flow, P[...,0], R[:,:,0])

In [ ]:
def estimate2(P, R, log2_search_area, n_levels, w_side=17):
    if log2_search_area > 0:
        P_pyramid = LP.analyze(P, log2_search_area)
        R_pyramid = LP.analyze(R, log2_search_area)
    else:
        P_pyramid = [P]
        R_pyramid = [R]
    #total_flow = np.zeros(((R_pyramid[0].shape[0]+1)//2, (R_pyramid[0].shape[1]+1)//2, 2), dtype=np.int16)
    flow = np.zeros((R_pyramid[0].shape[0], R_pyramid[0].shape[1], 2), dtype=np.int16)
    for l in range(n_levels):
        #_R_pyramid = motion.make_prediction(R_pyramid[l], total_flow.astype(np.float32))
        #show_frame(_R_pyramid, f"R_pyramid[{l}]")
        #show_frame(R_pyramid[l], f"R_pyramid[{l}]")
        #show_frame(P_pyramid[l], f"P_pyramid[{l}]")
        #flow = spiral_estimate2(flow, P_pyramid[l], R_pyramid[l], window_side = w_side >> (log2_search_area - l))
        w_side >>= (log2_search_area - l)
        if w_side < 15:
            w_side = 15
        flow = spiral_estimate2(flow, P_pyramid[l], R_pyramid[l], window_side = w_side)
        #total_flow *= 2
        #total_flow += flow
        #total_flow = total_flow.astype(np.int16)
        #for y in range(total_flow.shape[0]):
        #    for x in range(total_flow.shape[1]):
        #        print(total_flow[y,x], end=' ')
        #    print()
        #print(80*"=")
        #show_frame(R_pyramid[l+1])
        R_pyramid[l+1] = LP.synthesize_step(R_pyramid[l], R_pyramid[l+1])
        P_pyramid[l+1] = LP.synthesize_step(P_pyramid[l], P_pyramid[l+1])
        flow = LP.synthesize_step(flow.astype(np.float32), np.zeros(flow.shape)).astype(np.int16)
        flow *= 2
    #_R = motion.make_prediction(R, total_flow.astype(np.float32))
    #show_frame(R, f"_R")
    #show_frame(P, f"P")
    flow = spiral_estimate2(flow, P, R, window_side = w_side)
    #total_flow = total_flow.astype(np.int16)
    return flow

In [ ]:
R = frame.read("../sequences/moving_circles/", 0)
P = frame.read("../sequences/moving_circles/", 1)

In [ ]:
show_frame(R); show_frame(P)

In [ ]:
np.set_printoptions(linewidth=120)
flow = estimate2(P[...,0], R[...,0], 3, 3, w_side=31)

In [ ]:
show_frame(flow[...,0])

In [ ]:
show_vectors(flow[::2, ::2])

In [ ]:
show_frame(flow[...,1])

In [ ]:
prediction = motion.make_prediction(R, flow.astype(np.float32))

In [ ]:
show_frame(prediction)

In [ ]:
show_frame(P)

In [ ]:
show_frame(R)

In [ ]:
show_frame(prediction-P)

## Ahora probamos con un desplazamiento de 2

In [ ]:
R = YUV.from_RGB(frame.read("../sequences/moving_circle/", 0).astype(np.int16))
P = YUV.from_RGB(frame.read("../sequences/moving_circle/", 2).astype(np.int16))

In [ ]:
#flow = estimate2(P[...,0], R[...,0], 1, 1, w_side=33)
flow = estimate2(P[...,0], R[...,0], 3, 3, w_side=33)

In [ ]:
prediction = motion.make_prediction(R, flow.astype(np.float32))

In [ ]:
show_frame(prediction, "Prediction")
show_frame(P, "Predicted")
show_frame(prediction-P)

## Ahora probamos con un desplazamiento de 3

In [ ]:
R = YUV.from_RGB(frame.read("../sequences/moving_circle/", 0).astype(np.int16))
P = YUV.from_RGB(frame.read("../sequences/moving_circle/", 3).astype(np.int16))

In [ ]:
flow = estimate2(P[...,0], R[...,0], 2, 2, w_side=33)

In [ ]:
prediction = motion.make_prediction(R, flow.astype(np.float32))

In [ ]:
show_frame(prediction, "Prediction")
show_frame(P, "Predicted")
show_frame(prediction-P)

## Ahora probamos con un desplazamiento de 4

In [ ]:
R = YUV.from_RGB(frame.read("../sequences/moving_circle/", 0).astype(np.int16))
P = YUV.from_RGB(frame.read("../sequences/moving_circle/", 4).astype(np.int16))

In [ ]:
flow = estimate2(P[...,0], R[...,0], 3, 3, w_side=33)

In [ ]:
prediction = motion.make_prediction(R, flow.astype(np.float32))

In [ ]:
show_frame(prediction, "Prediction")
show_frame(P, "Predicted")
show_frame(prediction-P)

## Más imágenes reales

In [ ]:
R = YUV.from_RGB(frame.read("../sequences/bus/", 0).astype(np.int16))
P = YUV.from_RGB(frame.read("../sequences/bus/", 1).astype(np.int16))

In [ ]:
flow = estimate2(P[...,0], R[...,0], 5, 5, w_side=65)

In [ ]:
initial_flow = np.zeros((P.shape[0], P.shape[1], 2), dtype=np.float32)
OF_flow = motion.estimate(P[...,0], R[...,0], initial_flow=initial_flow )

In [ ]:
show_vectors(flow[::8, ::8])

In [ ]:
show_vectors(OF_flow[::8, ::8])

In [ ]:
R = frame.read("../sequences/bus/", 0)
P = frame.read("../sequences/bus/", 1)
prediction = motion.make_prediction(R, flow.astype(np.float32))

In [ ]:
show_frame(prediction, "Prediction Mío")
show_frame(P, "Predicted")
show_frame(motion.make_prediction(R, OF_flow), "OF")

In [ ]:
frame.write(prediction, "/tmp/", 1)
frame.write(R, "/tmp/", 2)
frame.write(motion.make_prediction(R, OF_flow), "/tmp/", 3)

In [ ]:
frame.write(prediction, "/tmp/p", 0)

In [ ]:
frame.write(motion.make_prediction(R, OF_flow), "/tmp/pp", 0)

## Imágenes reales

In [ ]:
R = YUV.from_RGB(frame.read("../sequences/mobile/", 0).astype(np.int16))
P = YUV.from_RGB(frame.read("../sequences/mobile/", 1).astype(np.int16))

In [ ]:
flow = estimate2(P[...,0], R[...,0], 3, 3, w_side=33)

In [ ]:
initial_flow = np.zeros((P.shape[0], P.shape[1], 2), dtype=np.float32)
OF_flow = motion.estimate(P[...,0], R[...,0], initial_flow=initial_flow )

In [ ]:
R = frame.read("../sequences/mobile/", 0)
P = frame.read("../sequences/mobile/", 1)
prediction = motion.make_prediction(R, flow.astype(np.float32))

In [ ]:
show_frame(prediction, "Prediction")
show_frame(P, "Predicted")
show_frame((prediction-P)*10, "mio")
show_frame((R-P)*10, "sin")
show_frame((motion.make_prediction(R, OF_flow) - P)*10, "OF")

In [ ]:
frame.write(prediction-P, "/tmp/", 1)
frame.write(R-P, "/tmp/", 2)
frame.write(motion.make_prediction(R, OF_flow) - P, "/tmp/", 3)

In [ ]:
frame.write(prediction, "/tmp/p", 0)

In [ ]:
frame.write(motion.make_prediction(R, OF_flow), "/tmp/pp", 0)

## Brute force

In [ ]:
def spiral_estimate2(initial_flow, P, R, window_side=15):
    print("window_side =", window_side)
    print("inital_flow.dtype", initial_flow.dtype)
    print("R.shape", R.shape)
    print("initial_flow.shape", initial_flow.shape)
    extended_R = np.zeros((R.shape[0]+2, R.shape[1]+2))
    extended_R[1:R.shape[0]+1, 1:R.shape[1]+1] = motion.make_prediction(R, initial_flow)[:R.shape[0], :R.shape[1]]
    show_frame(R, "R")
    show_frame(extended_R[1:R.shape[0]+1, 1:R.shape[1]+1], "extended_R")
    show_frame(P, "P")
    D = np.zeros((P.shape[0], P.shape[1], 9), dtype=np.float32)
    for i in range(9):
        diff = extended_R[spiral_[i][1] : R.shape[0] + spiral_[i][1],
                          spiral_[i][0] : R.shape[1] + spiral_[i][0]] - P
        #print(diff)
        D[:,:,i] = abs(diff)#diff*diff
    # Tenemos la distancia L2, pixel-wise, para cada posible desplazamiento de ambas matrices, R y P.
    # Ahora se trata de encontrar una nueva matriz de distancias donde se tenga en cuenta las distancias adyacentes.
    DD = np.empty_like(D).astype(np.float32)
    DD[...] = D
    #kernel = np.ones((window_side,window_side),np.float32)/(window_side*window_side)
    #D = cv.filter2D(DD, -1, kernel).astype(np.int)
    D = cv.GaussianBlur(DD, (window_side, window_side), 0).astype(np.int)
    #show_frame(D[...,0]+D[...,1])
    #show_frame(D[...,0])
    #for y in range(D.shape[0]):
    #    for x in range(D.shape[1]):
    #        print(D[y,x], end=' ')
    #    print()
    #print(80*"=")
    #for i in range(9):
    #    print(D[...,i])
    flow_indexes = np.argmin(D, axis=2)
    #for i in range(flow_indexes.shape[0]):
    #    for j in range(flow_indexes.shape[1]):
    #        print(flow_indexes[i,j], end='')
    #    print()
    #print(80*"=")
    #print(spiral[flow_indexes].shape)
    #print(initial_flow.shape)
    flow = initial_flow[:flow_indexes.shape[0], :flow_indexes.shape[1]] + spiral[flow_indexes]
    return flow

In [ ]:
R = YUV.from_RGB(frame.read("../sequences/moving_circle/", 0).astype(np.int16))[...,0]
P = YUV.from_RGB(frame.read("../sequences/moving_circle/", 4).astype(np.int16))[...,0]
#show_frame(R)

In [ ]:
search_area_side = 8
window_side = 17
extended_R = np.zeros((R.shape[0] + search_area_side, R.shape[1] + search_area_side))
extended_R[search_area_side//2:R.shape[0]+search_area_side//2,
           search_area_side//2:R.shape[1]+search_area_side//2] = R
show_frame(extended_R)
flow = np.zeros((P.shape[0], P.shape[1], 2), dtype=np.int8)
#print(flow.shape)
min_error = np.full((P.shape[0], P.shape[1]), 255, dtype=np.uint8)
#print(min_error.max())
for y in range(search_area_side):
    for x in range(search_area_side):
        error = extended_R[y:P.shape[0] + y,
                           x:P.shape[1] + x] - P
        a_error = abs(error)
        blur_a_error = cv.GaussianBlur(a_error, (window_side, window_side), 0).astype(np.int)
        #print(blur_a_error.max())
        which_min = blur_a_error <= min_error
        #print(which_min.shape)
        #where_min = np.argmin(np.array([min_error, blur_a_error]), axis=0)
        #flow = np.where(which_min, flow[y, x], (x, y))
        #flow = np.where(which_min, flow, 1)
        flow[:,:,0] = np.where(which_min, x-search_area_side//2, flow[:,:,0])
        flow[:,:,1] = np.where(which_min, y-search_area_side//2, flow[:,:,1])
        #print(where_min.shape)
        #print(y, x, where_min)
        min_error = np.minimum(min_error, blur_a_error)
        #print(min_error.shape)
        #show_frame(where_min, f"{y} {x}")
        #show_frame(min_error, f"min {y} {x}")
        #show_frame(blur_a_error, f"blur {y} {x}")
#x = y = search_area_side//2
#error = extended_R[y:P.shape[0] + y, x:P.shape[1] + x] - P
#a_error = abs(error)
#blur_a_error = cv.GaussianBlur(a_error, (window_side, window_side), 0).astype(np.int)
#show_frame(blur_a_error, f"blur {0} {0}")
#which_min = blur_a_error <= min_error
#print("max =", min_error.max())
#flow[:,:,0] = np.where(which_min, x-search_area_side//2, flow[:,:,0])
#flow[:,:,1] = np.where(which_min, y-search_area_side//2, flow[:,:,1])

#show_vectors(flow[::8, ::8])
show_vectors(flow[::1, ::1])
print(flow.min())

In [ ]:
R = frame.read("../sequences/moving_circle/", 0)
P = frame.read("../sequences/moving_circle/", 4)
prediction = motion.make_prediction(R, flow.astype(np.float32))
show_frame(prediction)
show_frame(P-prediction)

In [ ]:
def full_search_motion_estimation(predicted, reference, search_range, overlapping_area_side):
    extended_reference = np.zeros((R.shape[0] + search_range, R.shape[1] + search_range))
    extended_reference[search_range//2 : reference.shape[0]+search_range//2,
                       search_range//2 : reference.shape[1]+search_range//2] = reference
    flow = np.zeros((predicted.shape[0], predicted.shape[1], 2), dtype=np.int8)
    min_error = np.full((predicted.shape[0], predicted.shape[1]), 255, dtype=np.uint8)
    for y in range(search_range):
        print(f"{y}/{search_range-1}", end='\r')
        for x in range(search_range):
            error = extended_reference[y : predicted.shape[0] + y,
                                       x : predicted.shape[1] + x] - predicted
            a_error = abs(error)
            blur_a_error = cv.GaussianBlur(a_error, (overlapping_area_side, overlapping_area_side), 0).astype(np.int)
            which_min = blur_a_error <= min_error
            flow[:,:,0] = np.where(which_min, x-search_range//2, flow[:,:,0])
            flow[:,:,1] = np.where(which_min, y-search_range//2, flow[:,:,1])
            min_error = np.minimum(min_error, blur_a_error)
    return flow

In [ ]:
R = YUV.from_RGB(frame.read("../sequences/bus/", 0).astype(np.int16))
P = YUV.from_RGB(frame.read("../sequences/bus/", 1).astype(np.int16))

In [ ]:
R = cv.pyrUp(R)
P = cv.pyrUp(P)

In [ ]:
print(R.shape, P.shape)

In [ ]:
flow = full_search_motion_estimation(P[...,0], R[...,0], 32, 17)

In [ ]:
print(flow.shape)

In [ ]:
R = frame.read("../sequences/bus/", 0)
R = cv.pyrUp(R)
P = frame.read("../sequences/bus/", 1)
P = cv.pyrUp(P)
prediction = motion.make_prediction(R, flow.astype(np.float32))
show_frame(prediction, "Prediction")
show_frame(P, "Predicted")
show_frame(normalize(prediction-P), "error")

In [ ]:
R = YUV.from_RGB(frame.read("../sequences/bus/", 0).astype(np.int16))[...,0]
P = YUV.from_RGB(frame.read("../sequences/bus/", 1).astype(np.int16))[...,0]
OF_flow = motion.estimate(P.astype(np.float32), R.astype(np.float32),
                          initial_flow=np.zeros((P.shape[0], P.shape[1], 2), dtype=np.float32),
                          levels=3, wside=200, iters=1)
R = frame.read("../sequences/bus/", 0)
P = frame.read("../sequences/bus/", 1)
prediction = motion.make_prediction(R, OF_flow.astype(np.float32))
show_frame(prediction, "Prediction")
show_frame(P, "Predicted")
show_frame(normalize(prediction-P), "error")

In [ ]:
        #min_error = np.argmin([min_error, blur_a_error], axis=2)
        show_frame(blur_a_error, f"{y} {x}")
        

In [ ]:
############################3

In [ ]:
D = []
for i in _spiral:
    difference = extended_R[i[0]:R.shape[0]+i[0], i[1]:R.shape[1]+i[1]] - P
    D.append(difference)

In [ ]:
D[1].max()